# DeckGL Point Cloud Layer Example

This notebook demonstrates how to visualize 3D point cloud data using DeckGL's PointCloudLayer integrated with MapLibre in anymap.

The PointCloudLayer is perfect for visualizing large sets of 3D points with color and normal information, such as LiDAR data, 3D scans, or any spatial point cloud dataset.

## Installation

```bash
pip install anymap
```

In [ ]:
from anymap.maplibre import MapLibreMap

## Example 1: Point Cloud from URL

This example loads a point cloud dataset from a remote URL. The dataset contains position, color, and normal information for each point.

In [ ]:
# Create a MapLibre map centered on San Francisco
m = MapLibreMap(
    center=[-122.4, 37.74],
    zoom=11,
    pitch=45,  # Tilt the map for better 3D view
    bearing=0,
    style="https://tiles.openfreemap.org/styles/bright",
)

# Add PointCloudLayer with data from URL
m.add_deckgl_layer(
    "pointcloud",
    "PointCloudLayer",
    "https://raw.githubusercontent.com/visgl/deck.gl-data/master/website/pointcloud.json",
    props={
        "getPosition": "position",
        "getColor": "color",
        "getNormal": "normal",
        "pointSize": 2,
        "coordinateOrigin": [-122.4, 37.74],
        "coordinateSystem": "COORDINATE_SYSTEM.METER_OFFSETS",
        "pickable": True,
        "opacity": 0.8,
    },
)

# Add layer control to toggle visibility
# m.add_layer_control()
m.to_html("pointcloud_example.html", title="Point Cloud Visualization")
m

## Example 2: Simple Point Cloud with Custom Data

Create a simple point cloud from scratch with custom data.

In [ ]:
import numpy as np

# Generate a simple 3D point cloud - a spiral
n_points = 1000
t = np.linspace(0, 4 * np.pi, n_points)

# Create spiral in meter offsets
x = 100 * np.cos(t) * (1 + t / (4 * np.pi))
y = 100 * np.sin(t) * (1 + t / (4 * np.pi))
z = 5 * t

# Color gradient from blue to red based on height
colors = np.zeros((n_points, 3))
colors[:, 0] = (t / t.max() * 255).astype(int)  # Red increases with height
colors[:, 2] = ((1 - t / t.max()) * 255).astype(int)  # Blue decreases with height

# Create data array
point_data = [
    {
        "position": [float(x[i]), float(y[i]), float(z[i])],
        "color": [int(colors[i, 0]), int(colors[i, 1]), int(colors[i, 2])],
        "normal": [0, 0, 1],  # Simple upward normals
    }
    for i in range(n_points)
]

# Create map
m2 = MapLibreMap(
    center=[-122.4, 37.74],
    zoom=13,
    pitch=60,
    bearing=30,
    style="https://tiles.openfreemap.org/styles/dark",  # Dark style for better contrast
)

# Add the spiral point cloud
m2.add_deckgl_layer(
    "spiral",
    "PointCloudLayer",
    point_data,
    props={
        "getPosition": "position",
        "getColor": "color",
        "getNormal": "normal",
        "pointSize": 3,
        "coordinateOrigin": [-122.4, 37.74],
        "coordinateSystem": "COORDINATE_SYSTEM.METER_OFFSETS",
        "pickable": True,
        "opacity": 0.9,
    },
)

m2.add_layer_control()

m2

## Example 3: Point Cloud Grid

Create a grid-based point cloud with varying heights.

In [ ]:
# Generate a grid of points with varying heights
grid_size = 30
spacing = 20  # meters between points

grid_data = []
for i in range(grid_size):
    for j in range(grid_size):
        x = (i - grid_size / 2) * spacing
        y = (j - grid_size / 2) * spacing
        # Create wave pattern for height
        z = 30 * np.sin(i / 3) * np.cos(j / 3) + 50

        # Color based on height
        height_norm = (z + 20) / 90  # Normalize to 0-1
        color = [
            int(255 * height_norm),
            int(100 + 155 * (1 - height_norm)),
            int(255 * (1 - height_norm)),
        ]

        grid_data.append(
            {
                "position": [float(x), float(y), float(z)],
                "color": color,
                "normal": [0, 0, 1],
            }
        )

# Create map with the grid
m3 = MapLibreMap(
    center=[-73.935242, 40.730610],  # New York
    zoom=14,
    pitch=70,
    bearing=45,
    style="https://tiles.openfreemap.org/styles/liberty",
)

m3.add_deckgl_layer(
    "grid",
    "PointCloudLayer",
    grid_data,
    props={
        "getPosition": "position",
        "getColor": "color",
        "getNormal": "normal",
        "pointSize": 4,
        "coordinateOrigin": [-73.935242, 40.730610],
        "coordinateSystem": "COORDINATE_SYSTEM.METER_OFFSETS",
        "pickable": True,
        "material": {
            "ambient": 0.5,
            "diffuse": 0.8,
            "shininess": 32,
            "specularColor": [255, 255, 255],
        },
    },
)

m3.add_layer_control()

m3

## Example 4: Multiple Point Cloud Layers

Demonstrate multiple point cloud layers on the same map with different colors and sizes.

In [ ]:
# Create a map
m4 = MapLibreMap(
    center=[0, 0],
    zoom=12,
    pitch=55,
    bearing=0,
    style="https://tiles.openfreemap.org/styles/positron",
)

# First point cloud - red sphere
n_sphere = 500
phi = np.random.uniform(0, 2 * np.pi, n_sphere)
theta = np.random.uniform(0, np.pi, n_sphere)
radius = 200

sphere_data = [
    {
        "position": [
            float(radius * np.sin(theta[i]) * np.cos(phi[i]) - 300),
            float(radius * np.sin(theta[i]) * np.sin(phi[i])),
            float(radius * np.cos(theta[i]) + 100),
        ],
        "color": [255, 50, 50],
        "normal": [
            float(np.sin(theta[i]) * np.cos(phi[i])),
            float(np.sin(theta[i]) * np.sin(phi[i])),
            float(np.cos(theta[i])),
        ],
    }
    for i in range(n_sphere)
]

# Second point cloud - blue sphere
sphere_data2 = [
    {
        "position": [
            float(radius * np.sin(theta[i]) * np.cos(phi[i]) + 300),
            float(radius * np.sin(theta[i]) * np.sin(phi[i])),
            float(radius * np.cos(theta[i]) + 100),
        ],
        "color": [50, 50, 255],
        "normal": [
            float(np.sin(theta[i]) * np.cos(phi[i])),
            float(np.sin(theta[i]) * np.sin(phi[i])),
            float(np.cos(theta[i])),
        ],
    }
    for i in range(n_sphere)
]

# Add both layers
m4.add_deckgl_layer(
    "red-sphere",
    "PointCloudLayer",
    sphere_data,
    props={
        "getPosition": "position",
        "getColor": "color",
        "getNormal": "normal",
        "pointSize": 3,
        "coordinateOrigin": [0, 0],
        "coordinateSystem": "COORDINATE_SYSTEM.METER_OFFSETS",
        "pickable": True,
        "material": {"ambient": 0.4, "diffuse": 0.8, "shininess": 32},
    },
)

m4.add_deckgl_layer(
    "blue-sphere",
    "PointCloudLayer",
    sphere_data2,
    props={
        "getPosition": "position",
        "getColor": "color",
        "getNormal": "normal",
        "pointSize": 3,
        "coordinateOrigin": [0, 0],
        "coordinateSystem": "COORDINATE_SYSTEM.METER_OFFSETS",
        "pickable": True,
        "material": {"ambient": 0.4, "diffuse": 0.8, "shininess": 32},
    },
)

# Add layer control to toggle individual layers
m4.add_layer_control()

m4

## Controlling Point Cloud Layers

You can control the visibility of point cloud layers programmatically:

In [ ]:
# Toggle visibility of the red sphere
m4.set_deckgl_layer_visibility("red-sphere", False)

# Show it again
m4.set_deckgl_layer_visibility("red-sphere", True)

# Get information about current DeckGL layers
print("Current DeckGL layers:", list(m4.get_deckgl_layers().keys()))

## Export to HTML

You can export any map with point cloud layers to a standalone HTML file:

In [ ]:
# Export the multi-sphere example
m4.to_html("pointcloud_example.html", title="Point Cloud Visualization")
print("Map exported to pointcloud_example.html")

## Key Parameters for PointCloudLayer

**Basic Properties:**
- **data**: Point cloud data as array of objects, GeoJSON, or URL to LAZ/other formats
- **getPosition**: Accessor for point positions (x, y, z) - not needed for LAZ files
- **getColor**: Accessor for point colors (RGB or RGBA) - not needed for LAZ files
- **getNormal**: Accessor for surface normals (for lighting) - can be constant for LAZ

**Rendering Properties:**
- **pointSize**: Size of each point in pixels or meters
- **sizeUnits**: 'pixels' (default) or 'meters'
- **opacity**: Layer opacity (0-1)
- **pickable**: Whether points can be selected/clicked

**Coordinate Systems:**
- **LNGLAT**: Geographic coordinates (longitude, latitude) - use for LAZ files
- **METER_OFFSETS**: Local meter offsets from origin - use for custom data
- **coordinateOrigin**: Origin point for METER_OFFSETS system

**Material Properties (for 3D lighting):**
- **ambient**: Ambient light factor (0-1)
- **diffuse**: Diffuse light factor (0-1)  
- **shininess**: Surface shininess (0-128)
- **specularColor**: Color of specular highlights [R, G, B]

**LAZ-Specific:**
When loading LAZ files, you typically only need:
```python
props={
    "getNormal": [0, 0, 1],
    "pointSize": 1,
    "coordinateSystem": "COORDINATE_SYSTEM.LNGLAT",
    "material": {...}  # Optional
}
```

For more details, see:
- [DeckGL PointCloudLayer documentation](https://deck.gl/docs/api-reference/layers/point-cloud-layer)
- [Loaders.gl LAS/LAZ loader](https://loaders.gl/modules/las/docs/api-reference/las-loader)

## LAZ Format Notes

**What is LAZ?**
- LAZ is a compressed version of the LAS format (LASer)
- Commonly used for LiDAR point cloud data
- Can contain millions of points with position, color, intensity, and classification data
- Typical file sizes: 10-100MB compressed (100s of MB uncompressed)

**How DeckGL Handles LAZ:**
- DeckGL uses `@loaders.gl/las` library (included in deck.gl distribution)
- The loader runs in a Web Worker for non-blocking loading
- Automatically extracts position, color, and other attributes
- Supports streaming for large files

**Performance Tips:**
- LAZ files with millions of points may take 10-60 seconds to load
- Consider using `pointSize: 1` for dense datasets
- Adjust `opacity` if points overlap significantly
- Use `material` properties for better 3D visualization

**Common Issues:**
- **CORS errors**: Ensure LAZ file is served with proper CORS headers
- **File not loading**: Check browser console for errors
- **Slow loading**: Large files may need time; check network tab in DevTools
- **Memory issues**: Very large files (>200MB) may cause browser slowdown

## Using Local LAZ Files

If you have a local LAZ file (like the Autzen dataset), you can serve it and use it in your map. Here's how:

1. **Serve the file locally** using Python's built-in HTTP server:
```bash
# In the directory containing your LAZ file
python -m http.server 8000
```

2. **Reference the local file** in your map:
```python
m_local = MapLibreMap(center=[-123.075, 44.05], zoom=16, pitch=60)
m_local.add_deckgl_layer(
    "local-lidar",
    "PointCloudLayer",
    "http://localhost:8000/autzen.laz",  # Your local LAZ file
    props={
        "getNormal": [0, 0, 1],
        "pointSize": 1,
        "coordinateSystem": "COORDINATE_SYSTEM.LNGLAT",
        "pickable": True,
    }
)
```

**Alternative**: Use publicly available LAZ datasets:
- USGS 3DEP LiDAR: https://www.usgs.gov/3d-elevation-program
- OpenTopography: https://opentopography.org/
- Various city open data portals often provide LiDAR in LAZ format

In [ ]:
# Create a map for LAZ point cloud visualization
m5 = MapLibreMap(
    center=[-123.075, 44.05],  # Autzen Stadium area, Eugene, Oregon
    zoom=16,
    pitch=60,
    bearing=0,
    style="https://tiles.openfreemap.org/styles/liberty",
)

# Add PointCloudLayer with LAZ data
# The LAZ file will be loaded using @loaders.gl/las which DeckGL includes
# This is the Autzen Stadium LiDAR dataset from Kaarta
m5.add_deckgl_layer(
    "lidar-laz",
    "PointCloudLayer",
    "https://raw.githubusercontent.com/visgl/deck.gl-data/master/examples/point-cloud-laz/indoor.0.1.laz",
    props={
        # For LAZ files loaded by loaders.gl, we don't specify accessors
        # The loader automatically provides position and color data
        "getNormal": [0, 0, 1],  # Default normal vector
        "pointSize": 1,
        "sizeUnits": "pixels",
        # LAZ files use absolute coordinates, not offset coordinates
        "coordinateSystem": "COORDINATE_SYSTEM.LNGLAT",
        "pickable": True,
        "opacity": 1.0,
        # Material properties for better 3D appearance
        "material": {
            "ambient": 0.35,
            "diffuse": 0.6,
            "shininess": 32,
            "specularColor": [60, 64, 70],
        },
    },
)

m5.add_layer_control()

print("LAZ file will be loaded automatically by DeckGL's loaders.gl")
print("This may take a moment depending on file size and network speed")

m5

## Example 5: LiDAR Point Cloud from LAZ Format

This example demonstrates loading LiDAR data from a LAZ (LASzip compressed) file. LAZ is a compressed format commonly used for LiDAR point cloud data.

**Note**: For LAZ support in web browsers, DeckGL uses the `@loaders.gl/las` loader which is loaded automatically. The LAZ file should be accessible via URL.

For this example, we'll use a publicly available LAZ dataset. You can also use local LAZ files by serving them with a local web server.